# DOES POSE ESTIMATION and OBJECT DETECTION and also display TOTAL OBJECTS in the video frame(WORKS) but FCAE RECOGNITION (DOESNOT WORKS)

In [2]:
import cv2
import face_recognition
import mediapipe as mp
from ultralytics import YOLO
import os

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')

# Initialize Mediapipe Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture
input_video = r'E:\jupyter_notebook\Pictures\trump1.mp4'
output_video = r'E:\jupyter_notebook\Pictures\trump1_out_combined101_with_tota_object_count.mp4'
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Load known faces
known_face_encodings = []
known_face_names = []
known_faces_dir = r'E:\jupyter_notebook\Music\FOLDER_FOR_FACE_RECOGNITION\TRUMP'  # Folder containing known face images

for image_name in os.listdir(known_faces_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(known_faces_dir, image_name)
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)
        if face_encodings:  # Check if at least one face was found in the image
            known_face_encodings.append(face_encodings[0])
            known_face_names.append(os.path.splitext(image_name)[0])  # Use filename without extension as the name
        else:
            print(f"Warning: No face found in {image_name}. Skipping this image.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]  # Convert BGR to RGB
    
    # Perform face recognition
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Check if the face matches any known face
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        # Use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        # Label the face
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    # Perform object detection
    results = model(frame)
    object_count = 0  # Initialize object count
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            class_id = int(box.cls[0].item())
            class_name = result.names.get(class_id, 'Unknown')
            confidence = box.conf[0].item()
            label = f'{class_name}: {confidence:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            object_count += 1  # Increment object count
    
    # Display the total number of objects
    cv2.putText(frame, f'Total Objects: {object_count}', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Perform pose estimation
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # Show the output frame
    cv2.imshow('Frame', frame)
    
    # Write the output video frame
    out.write(frame)
    
    # Press 'q' to exit the video window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x384 2 persons, 272.3ms
Speed: 8.5ms preprocess, 272.3ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 272.5ms
Speed: 5.8ms preprocess, 272.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 1 clock, 286.3ms
Speed: 5.1ms preprocess, 286.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bench, 2 ties, 275.0ms
Speed: 5.0ms preprocess, 275.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 277.7ms
Speed: 5.0ms preprocess, 277.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 317.4ms
Speed: 5.0ms preprocess, 317.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 tie, 272.6ms
Speed: 5.0ms preprocess, 272.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bench, 1 tie, 2

# WORKS
# DOES OBJECT DETECTION, POSE ESTIMATION , TOTAL COUNTS, ADDDS AUDIO

In [8]:
import cv2
import face_recognition
import mediapipe as mp
from ultralytics import YOLO
import os
from moviepy.editor import VideoFileClip

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')

# Initialize Mediapipe Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture
input_video = r'E:\jupyter_notebook\Pictures\trump1.mp4'
temp_output_video = r'E:\jupyter_notebook\Pictures\temp_output_video.mp4'
output_video = r'E:\jupyter_notebook\Pictures\trump1_out_combined101_with_tota_object_count2.mp4'
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer with a different codec
fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Try 'MJPG' or 'mp4v'
out = cv2.VideoWriter(temp_output_video, fourcc, fps, (width, height))

# Load known faces
known_face_encodings = []
known_face_names = []
known_faces_dir = r'E:\jupyter_notebook\Music\FOLDER_FOR_FACE_RECOGNITION\TRUMP'

for image_name in os.listdir(known_faces_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(known_faces_dir, image_name)
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)
        if face_encodings:
            known_face_encodings.append(face_encodings[0])
            known_face_names.append(os.path.splitext(image_name)[0])
        else:
            print(f"Warning: No face found in {image_name}. Skipping this image.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]  # Convert BGR to RGB
    
    # Perform face recognition
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    results = model(frame)
    object_count = 0
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            class_id = int(box.cls[0].item())
            class_name = result.names.get(class_id, 'Unknown')
            confidence = box.conf[0].item()
            label = f'{class_name}: {confidence:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            object_count += 1
    
    cv2.putText(frame, f'Total Objects: {object_count}', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    cv2.imshow('Frame', frame)
    out.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Combine original audio with the processed video
from moviepy.editor import VideoFileClip

try:
    video_clip = VideoFileClip(temp_output_video)
    audio_clip = VideoFileClip(input_video).audio
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_video, codec='libx264')
except Exception as e:
    print(f"An error occurred: {e}")

# Remove the temporary video file
os.remove(temp_output_video)


0: 640x384 2 persons, 421.1ms
Speed: 13.2ms preprocess, 421.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 227.1ms
Speed: 6.1ms preprocess, 227.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 1 clock, 212.3ms
Speed: 4.5ms preprocess, 212.3ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bench, 2 ties, 221.9ms
Speed: 5.2ms preprocess, 221.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 209.1ms
Speed: 4.0ms preprocess, 209.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 ties, 204.8ms
Speed: 3.8ms preprocess, 204.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 tie, 268.6ms
Speed: 8.3ms preprocess, 268.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bench, 1 tie, 

MoviePy - Done.
Moviepy - Writing video E:\jupyter_notebook\Pictures\trump1_out_combined101_with_tota_object_count2.mp4



Moviepy - Done !
Moviepy - video ready E:\jupyter_notebook\Pictures\trump1_out_combined101_with_tota_object_count2.mp4
